In [1]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")

Reusing dataset cnn_dailymail (/home/jvidakovic/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
from transformers import PegasusTokenizer

model_name = "google/pegasus-large"
tokenizer = PegasusTokenizer.from_pretrained(model_name)

In [4]:
from utils import prepare_dataset

train_dataset = prepare_dataset(
    dataset["train"]["article"],
    dataset["train"]["highlights"],
    tokenizer=tokenizer
)

ModuleNotFoundError: No module named 'utils'